In [9]:
import torch
torch.set_default_tensor_type(torch.FloatTensor) 
import copy

from rct_data_generator import *
from outcome_models import *
from plotting_functions import *
from mcmc_bayes_update import *
from eig_comp_utils import *
from research_exp_utils import *
from rct_data_generator import generate_host_and_mirror


In [15]:
n_host_sample = 100 
sigma_error = 1 
d = 10 
A = torch.randn((d,d))
A = 1/(torch.det(A)) * A

T_allocation_host = torch.randn(d)
T_allocation_host = 100/torch.norm(T_allocation_host)*T_allocation_host

mu_nc = torch.randn(d)
mu_nc = 1/torch.norm(mu_nc)*mu_nc

mu_c = torch.randn(d)
mu_c = 1/torch.norm(mu_c)*mu_c

mu = torch.concat([mu_nc,mu_c])

In [16]:


X_host = (torch.randn((n_host_sample,d)) @ A ) 
T_host = torch.bernoulli(torch.sigmoid(X_host@ T_allocation_host))
X_host_times_T = (T_host.unsqueeze(dim=0).T * X_host)
X_host = torch.concat([X_host,X_host_times_T],dim=1)

Y_host = X_host @ mu
Y_host = (1/Y_host.norm()) * Y_host + sigma_error * torch.randn_like(Y_host)


In [17]:
prior_mean = torch.zeros(2 * d)
beta_0, sigma_0_sq,inv_cov_0 = prior_mean, sigma_error,torch.eye(2*d)
prior_hyperparameters = {'beta_0': beta_0, 'sigma_0_sq': sigma_0_sq,"inv_cov_0":inv_cov_0}
bayesian_regression = BayesianLinearRegression(prior_hyperparameters)
bayesian_regression.set_causal_index(10)

In [18]:
bayesian_regression.fit(X_host,Y_host)

{'posterior_mean': tensor([-1.1898e-03,  1.2456e-02,  1.8201e-02,  1.9689e-02, -5.6107e-03,
         -8.3676e-03, -7.1791e-03, -6.8800e-03, -1.5144e-02, -2.9510e-03,
          2.7700e-03,  5.8197e-03,  1.5529e-02,  9.8763e-03,  3.0067e-03,
         -6.8083e-03, -4.2633e-03,  5.1620e-05, -9.7203e-03,  4.2627e-03]),
 'posterior_cov_matrix': tensor([[ 9.9984e-01, -1.5437e-05,  1.9514e-05, -1.4198e-05,  3.4872e-05,
           6.8778e-06,  5.5239e-05, -2.0194e-05, -4.3888e-05,  1.8506e-05,
          -5.0210e-05, -7.4796e-06,  3.9109e-06, -1.7166e-05,  1.1786e-05,
           3.8964e-07,  2.9746e-05, -1.1926e-05, -1.7158e-05,  1.1564e-05],
         [-1.5437e-05,  9.9995e-01, -4.2014e-05, -2.4395e-05,  4.8470e-06,
           1.6644e-05,  3.2878e-05,  2.8497e-05,  1.7093e-05, -2.8572e-06,
          -7.4791e-06, -2.4057e-05, -2.4320e-05, -7.4754e-06,  2.5858e-06,
           9.8858e-06,  1.5040e-05,  1.7153e-05,  1.0240e-05, -3.0801e-06],
         [ 1.9514e-05, -4.2014e-05,  9.9984e-01, -6.4041e-

In [19]:
bayesian_regression.closed_form_obs_EIG(X_host),bayesian_regression.closed_form_causal_EIG(X_host)

(0.0007901837674741958, 0.00026653940438354856)